In [16]:
from constants import FileDirectory, StravaAPI
from utility import redis_manager, s3_manager
from utility.file_manager import FileManager

In [17]:
file_manager = FileManager()
strava_data = file_manager.load_file(FileDirectory.RAW_DATA_PATH, StravaAPI.DATA_KEY)
strava_data.info()

[2024-07-05 17:45:19] [INFO]  Successfully loaded file from /Users/hadid/GitHub/ETL/data/raw_data/strava_activity_data.xlsx


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246 entries, 0 to 1245
Data columns (total 94 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   resource_state                                   1246 non-null   int64  
 1   name                                             1246 non-null   object 
 2   distance                                         1246 non-null   float64
 3   moving_time                                      1246 non-null   int64  
 4   elapsed_time                                     1246 non-null   int64  
 5   total_elevation_gain                             1246 non-null   float64
 6   type                                             1246 non-null   object 
 7   sport_type                                       1246 non-null   object 
 8   workout_type                                     120 non-null    float64
 9   id                            

In [18]:
# Extract IDs
activity_ids = strava_data["id"]
print(f"ID Data Type: {type(activity_ids)} \n")
print(f"ID Sample: \n {activity_ids.head(3)}")
print(f"ID Count: {activity_ids.count()} \n")

ID Data Type: <class 'pandas.core.series.Series'> 

ID Sample: 
 0    11715123516
1    11714332345
2    11714332359
Name: id, dtype: int64
ID Count: 1246 



In [19]:
# Cache IDs
redis_manager.update_cached_ids(StravaAPI.ID_KEY, activity_ids)

[2024-07-05 17:45:19] [INFO]  Updating 'strava_activity_ids' cache in Redis...
[2024-07-05 17:45:19] [INFO]  Successfully updated cached IDs, added 1246 new IDs


In [10]:
# Get cached IDs
cached_ids = redis_manager.get_cached_ids(StravaAPI.ID_KEY)
print(f"\nCount of cached IDs: {len(cached_ids)}\n")
print(f"ID Data Type: {type(cached_ids)}")

[2024-07-05 14:34:47] [INFO]  Fetching 'strava_activity_ids' cache from Redis...
[2024-07-05 14:34:47] [INFO]  Successfully fetched 1247 IDs



Count of cached IDs: 1247

ID Data Type: <class 'set'>


# Redis | Delete Values

In [2]:
# Delete cached IDs
activity_ids = [11802227853, 11809701909, 11802227882, 11802227944]
redis_manager.delete_cache_values(StravaAPI.ID_KEY, activity_ids)

[2024-07-05 15:16:23] [INFO]  Successfully deleted 4 values from the set


# S3

In [9]:
s3_manager.post_data_to_s3(strava_data, "strava_activity_data")

[2024-06-12 17:58:22] [INFO]  Starting upload to S3 bucket: etl-5h3gn2wqhzfd, data source: strava_activity_data
[2024-06-12 17:58:26] [INFO]  DataFrame uploaded to S3 bucket 'etl-5h3gn2wqhzfd' as 'strava/strava_activity_data_2024-06-12_17-58-22.csv'


True